## Тестовое задание для Bewise, Чупина Д.В.

Первым делом стоит загрузить нужные для работы библиотеки, а также таблицу с данными в формате датафрейма.

In [1]:
import pandas as pd
import pymorphy2
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, dictionary
from yargy.pipelines import caseless_pipeline

In [2]:
df = pd.read_csv(r"C:\Users\Эрвин\Documents\Machine Learning\TestTask\NLP\test_data.csv")

После этого имеет смысл оставить только те реплики, которые говорит менеджер, потому что в задании нет указаний для клиента. Также стоит добавить три новых столбца, все булевые и по умолчанию принимают значение **False** для всех строк: первый столбец показывает, есть ли в строке приветствие, второй показывает, есть ли в строке прощание, а третий -- есть ли и приветствие и прощание со стороны менеджера во всём отдельно взятом диалоге. После этого можно взглянуть на данные полностью -- их совсем немного, всего 201 строка.

In [3]:
df = df.loc[df['role'] == 'manager']
df = df.reset_index(drop = True)
df['greeting'] = False
df['farewell'] = False
df['insight'] = False

In [4]:
pd.set_option("display.max_rows", None)
df

,dlg_id,line_n,role,text,greeting,farewell,insight
0,0,1,manager,Алло здравствуйте,False,False,False
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,False
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,False,False,False
3,0,8,manager,Угу а на что вы обращаете внимание при выборе,False,False,False
4,0,11,manager,Что для вас приоритет,False,False,False
5,0,15,manager,Ну у вас срок заканчивается поэтому мы набрали...,False,False,False
6,0,29,manager,А так нет не только поэтому просто я обратила ...,False,False,False
7,0,34,manager,А если вы 19 являетесь то лучше то идти бесплатно,False,False,False
8,0,36,manager,Ага хорошо,False,False,False
9,0,45,manager,Индивидуальным поэтому не все то есть сотрудни...,False,False,False


Здесь были выбраны распространённые реплики с приветствием, которые ищутся по всем ответам менеджера, и в случае их нахождения выводятся внизу вместе с индексом в датафрейме. Также сразу те строки, в которых есть приветствие, помечаются как таковые, что будет полезно для последнего задания. 

In [5]:
RULE = caseless_pipeline([
    'добрый',
    'доброе утро',
    'утро доброе',
    'добрый день',
    'день добрый',
    'добрый вечер',
    'вечер добрый',
    'здравствуйте'    
])

df['greeting'] = False

parser = Parser(RULE)
text = list(df['text'])
for i in range(len(text)):
    line = text[i].lower()
    for match in parser.findall(line):
        if (df.iloc[i, 4] == False):
            print(i, line)
            df.iloc[i, 4] = True

0 алло здравствуйте
16 алло здравствуйте
45 алло здравствуйте
85 алло дмитрий добрый день
86 добрый меня максим зовут компания китобизнес удобно говорить


Для поиска же реплик с представлением себя было выбрано считать за таковую ту, где перед или после имени есть глагол "зовут", что используется в распространённом способе представления себя. Изначально была совершена попытка поиска имён через метод библиотеки Natasha, однако с этим возникли проблемы: поскольку это запись телефонного разговора, то все слова здесь, за исключением первого в строке, записаны прописными буквами, что мешает распознанию имени как имени, поскольку стоит оно не в начале предложения и начинается с прописной буквы. Была предпринята попытка поработать с этой библиотекой в том случае, когда все слова в тексте начинались с заглавной буквы, но это не увенчалось успехом -- слишком много случалось ложно-положительных результатов.

Можно было искать по именам менеджеров, однако, поскольку к заданию не даются отдельно данные с их именами, я предположила, что мы не должны заранее их знать, а собирать всевозможные имена было бы крайне времязатратно.

Поэтому пришлось выбрать именно этот подход, который, к сожалению, не всегда срабатывает: имеется, к примеру, способ представления как "это [имя менеджера]". Но такой вариант было сложно учесть, поскольку сочетание слова "это" и существительного слишком распространено в тексте в иных контекстах, а не только в представлении своего имени. Однако, учитывая и такой минус, всё же данный подход можно в целом считать рабочим.

Также для удобства сразу же выводятся и реплика целиком, и имя менеджера. Имя определяется как таковое, если до него либо идёт слово "зовут", либо слово "меня" (но при условии, что текущее слово не "зовут"). После этого найденное имя выводится сразу после реплики.

In [6]:
RULE = or_(
    rule(    
        and_(
            gram('NOUN'),
        ),
        gram('NOUN').optional().repeatable(),
        dictionary({
            'зовут'
        })
    ),
    rule(    
        dictionary({
            'зовут'
        }),
        and_(
            gram('NOUN'),
        ),
        gram('NOUN').optional().repeatable()
    )
)


parser = Parser(RULE)
text = list(df['text'])
for i in range(len(text)):
    line = text[i].lower()
    for match in parser.findall(line):
        print(i, line)
        prev_word = ''
        name_found = False
        for word in line.split():
            if ((prev_word == 'меня' or prev_word == 'зовут') and word != 'зовут' and name_found == False):
                print('Имя менеджера: ' + word)
                name_found = True
            prev_word = word

1 меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Имя менеджера: ангелина
17 меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
Имя менеджера: ангелина
46 меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
Имя менеджера: ангелина
86 добрый меня максим зовут компания китобизнес удобно говорить
Имя менеджера: максим


Название компании находится следующим образом: оно определяется как таковое, если до названия шло слово "компания". Также, поскольку название компании может состоять из нескольких слов, выводятся все существительные, идущие после слова "компания".

In [7]:
RULE = rule(    
    dictionary({
        'компания'
    }),    
    and_(
        gram('NOUN'),
    ),
    gram('NOUN').optional().repeatable()
)

parser = Parser(RULE)
for i in range(len(text)):
    line = text[i].lower()
    for match in parser.findall(line):
        temp = ''
        for _ in match.tokens:
            temp += _.value + ' '
        print(i, temp)

1 компания диджитал бизнес 
17 компания диджитал бизнес 
46 компания диджитал бизнес 
86 компания китобизнес 


Прощание определяется схожим образом, что и приветствие; и так же информация о том, что данная реплика является прощанием, заносится в отдельный столбик.

In [8]:
RULE = caseless_pipeline([
    'всего доброго',
    'хорошего дня',
    'хорошего вечера',
    'до свидания',
    'до понедельника',
    'до вторника',
    'до среды',
    'до четверга',
    'до пятницы',
])

df['farewell'] = False

parser = Parser(RULE)
text = list(df['text'])
for i in range(len(text)):
    line = text[i].lower()
    for match in parser.findall(line):
        if (df.iloc[i, 5] == False):
            print(i, line)
            df.iloc[i, 5] = True

15 всего хорошего до свидания
44 до свидания
106 угу все хорошо да понедельника тогда всего доброго
120 во вторник все ну с вами да тогда до вторника до свидания
200 ну до свидания хорошего вечера


Сейчас можно пройтись по всем строкам таблицы и посмотреть, есть ли в отдельно взятом диалоге и приветствие, и прощание. Если да, то **все** строки данного диалога помечаются как **True** в столбце insight.

In [9]:
gret = False
fare = False
dlg = 0
for i in range(len(df)):
    if (df.iloc[i,0] > dlg):
        dlg+=1
        gret = False
        fare = False
    if (df.iloc[i,4] == True):
        gret = True
    if (df.iloc[i,5] == True):
        fare = True
    if (gret and fare):
        for j in range(len(df)):
            if (df.iloc[j,0] == dlg):
                df.iloc[j,6] = True

In [10]:
pd.set_option("display.max_rows", None)
df

,dlg_id,line_n,role,text,greeting,farewell,insight
0,0,1,manager,Алло здравствуйте,True,False,True
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False,True
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...,False,False,True
3,0,8,manager,Угу а на что вы обращаете внимание при выборе,False,False,True
4,0,11,manager,Что для вас приоритет,False,False,True
5,0,15,manager,Ну у вас срок заканчивается поэтому мы набрали...,False,False,True
6,0,29,manager,А так нет не только поэтому просто я обратила ...,False,False,True
7,0,34,manager,А если вы 19 являетесь то лучше то идти бесплатно,False,False,True
8,0,36,manager,Ага хорошо,False,False,True
9,0,45,manager,Индивидуальным поэтому не все то есть сотрудни...,False,False,True
